# FBA Quant FE Session HW3

## Woohyuk Choi

In [36]:
# Import

import time
import numpy as np
import scipy
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd
import math as math
import random

## Problem 1


***
<span style="font-size:100%"> 

Build a $ 15 $ -period binomial model whose parameters should be calibrated to a 'Black-Scholes geometric Brownian motion (GBM) model' with: 

$T = 0.25$ years, $S_0 = 100$, $r = 2$%, $\sigma = 30$% and a dividend yield of $c = 1$%. 

Hint: Your binomial model should use a value of $u = 1.0395$. Now answer the following questions:

</span> 

***

### Solution of 1-(a),(b)


Compute the price of an American call/put option with strike $K = 110$ and maturity $T = 0.25$ years.

In [10]:
S0 = 100
K = 110
T = 0.25
r = 0.02
N = 15
u = 1.0395
d = 1/u
optiontype = 'P'



In [11]:
def american_tree(K, T, S0, r, N, u, d, optiontype='P'):
    dt = T/N
    q = (np.exp(r*dt) - d)/(u-d)
    disc = np.exp(-r*dt)

    S = np.zeros(N+1) 
    for j in range(0, N+1):
        S[j] = S0 * u**j * d**(N-j) 

    C = np.zeros(N+1)
    for j in range(0, N+1):
        if optiontype == 'P':
            C[j] = max(0, K-S[j])
        else : 
            C[j] = max(0,S[j]-K)

    for i in np.arange(N-1, -1, -1):
        for j in range(0, i+1):
            S = S0 * u**j * d**(i-j)
            C[j] = disc * (q*C[j+1] + (1-q)*C[j])
            if optiontype == 'P':
                C[j] = max(C[j], K-S)
            else:
                C[j] = max(C[j], S-K) 
    
    return C[0]


put = american_tree(K,T,S0,r,N,u,d,optiontype='P')
call = american_tree(K,T,S0,r,N,u,d,optiontype='C')

df=pd.DataFrame([[call, put]], columns=['call','put'])
df

,call,put
0,2.679125,12.227462


### Solution of 1-(c)

Is it ever optimal to early exercise the put option of part (b)?

No, early exercise never be optimal.

Let $E$: one American call $c$, $X e^{−r(T −t)}$ cash , $F$ : one share $S$ be a two different portfolio.

If the exercise time $\tau < T$ , the value of $E$ is 

$$E= (S-X) + Xe^{-r(T-t)} < S  = F$$ 

If the exercise time $\tau = T$ , the value of $E$ is

$$E = \max(S-X , 0) + X  > S = F$$

Shortly, because of time value.

### Solution fo 1-(e)

Do the call and put option prices of parts (a) and (b) satisfy put-call parity? Why or why not?

No, if it applies , there will be an arbitrage.

## Problem 2

***
Consider the following two period binomial tree. 

Suppose you purchase an European call option with maturity $T = 2$, Strike = $100$. Suppose $R_f = 1.0001$, that is, $1 invested at $T = 0$ is worth $1.0001^r$ at time $ t $. At each time step, the stock can increase by $a$ factor of $1.05$ or decrease by a factor of $ \frac {1}{1.05}$. 

Assume there is no arbitrage.
***

### Solution of 2 - (a)

Compute the price of the option.


In [24]:
u = 1.05
d = 1/u
rf = 1.0001

p=(rf-d)/(u-d)

s0 = 100
su = 100*u
sd = 100*d
suu = 100*u*u
sud = 100*u*d
sdd = 100*d*d

cuu = (suu-100)*p*p

if sud > 100:
    cud = (sud-100)*p*(1-p)
else :
    cud = 0

if sdd > 100:
    cdd = sdd*(1-p)*(1-p)
else : 
    cdd = 0

c_aftertwoyears = cuu + cud + cdd

c_present = c_aftertwoyears * ((1/rf)**2)



print('option price : ' , c_present)


option price :  2.4487892664393067


### Solution of 2-(b)


Since European Call option = American Call option , option price is same.

## Problem 3

***

Suppose that $u = e^w$ , where $w$ is normal with expected value $\bar{w}$ and variance $\sigma ^2$ . Then, 
$$\bar{u} = \frac 1 {\sqrt{2\pi \sigma^2}} \int_{-\infty}^{\infty} e^w e^{-\frac {(w-\bar{w})^2} {2\sigma^2}} dw $$

Show that 
$$ w - \frac {(w-\bar{w})^2} {2\sigma^2} = -\frac 1 {2\sigma^2} [w - (\bar{w} + \sigma^2)]^2 + \bar{w} + \frac {\sigma^2} 2$$

Use the fact that 
$$ \frac 1 {\sqrt{2\pi\sigma^2}} \int_{-\infty}^{\infty} e^{-\frac {(x-\bar{x})^2} {2\sigma^2}}dx = 1$$

to evaluate $\bar{u}$.

***

# Problem 4

### Solution of 4-(a)

In [27]:
grow=[1.6,1.5,1.4,1.3,1.2,1.15,1.1,1.05,1.02,1.01]

u = 1.2
d = 0.9
rf = 1.1

price = 2
s0 = 5

p=(rf-d)/(u-d)


print("Expected rate : " , p*u+(1-p)*d)




Expected rate :  1.1


As calculated above, expected rate is larger than $1$. Then, it is not good to cut.

### Solution of 4-(b)

In [33]:
N = 10
S0  = 5
T = 10
dt = T/N
r = 0.1
u = 1.2
d = 0.9
p = ( np.exp(r*dt) - d) / (u -d)

for k in reversed(range(N+1)):
    ST = S0 * u**k * d ** (N-k)
    print(round(ST-20,2))



10.96
3.22
-2.59
-6.94
-10.2
-12.65
-14.49
-15.87
-16.9
-17.68
-18.26


# Problem 5


$$p_k=p\cdot p_{k+1}+(1-p)\cdot p_{k-1}$$

$$(p_(k+1)-p_k)=((1-p)/p)(p_k-p_(k-1))$$


$$p_i=\sum_{k=0}^{i-1} (\frac q p)^k P_1$$

$$P_n=1$$



# Problem 6

Since solution of Monty Hall problem is so famous, so I introduce the simulation of this problem by quotting internet code websites. 

In [38]:
doors = [1, 2, 3]


[1, 2, 3]

In [40]:
import random

def monty_door_random(doors, alice_door, prize_door):
    monty_door = random.choice(doors)
    return monty_door

monty_door_random(doors, None, None)

1

In [42]:
N = 10000
result = [None]*N
for i in range(N):
    alice_door = random.choice(doors)
    prize_door = random.choice(doors)
    monty_door = monty_door_random(doors, alice_door, prize_door)
		

    while monty_door == prize_door:
         alice_door = random.choice(doors)
         prize_door = random.choice(doors)
         monty_door = monty_door_random(doors, alice_door, prize_door)
    remaining_doors = set(doors) - {alice_door} - {monty_door}
    result[i] = random.choice(list(remaining_doors)) == prize_door

print ("Probability of getting prize when Monty chooses the door randomly?\n %f" % (float(sum(result))/N))

Probability of getting prize when Monty chooses the door randomly?
 0.495500


In [44]:
def monty_door_avoid_alice(doors, alice_door, prize_door):
    candidates = list(set(doors) - {alice_door})
    monty_door = random.choice(candidates)
    return monty_door

result = [None]*N
for i in range(N):
     alice_door = random.choice(doors)
     prize_door = random.choice(doors)
     monty_door = monty_door_avoid_alice(doors, alice_door, prize_door)
     while monty_door == prize_door:
         alice_door = random.choice(doors)
         prize_door = random.choice(doors)
         monty_door = monty_door_avoid_alice(doors, alice_door, prize_door)
     
     remaining_doors = set(doors) - {alice_door} - {monty_door}
     result[i] = random.choice(list(remaining_doors)) == prize_door

print ("Probability of getting prize when Monty chooses the door avoiding Alice?\n %f" % (float(sum(result))/N))

Probability of getting prize when Monty chooses the door avoiding Alice?
 0.501300


In [45]:
def monty_door_avoid_both(doors, alice_door, prize_door):
    candidates = list(set(doors) - {prize_door, alice_door})
    monty_door = random.choice(candidates)
    return monty_door

result = [None]*N
for i in range(N):
     alice_door = random.choice(doors)
     prize_door = random.choice(doors)
     monty_door = monty_door_avoid_both(doors, alice_door, prize_door)
     remaining_doors = set(doors) - {alice_door} - {monty_door}
     result[i] = random.choice(list(remaining_doors)) == prize_door

print ("Probability of getting prize when Monty chooses the door avoiding both Alice and Prize?\n %f" % (float(sum(result))/N))

Probability of getting prize when Monty chooses the door avoiding both Alice and Prize?
 0.666900
